In [ ]:
import os
import shutil

# Get user profile path
user_profile = os.environ["USERPROFILE"]

# Paths to Hugging Face cached models
cached_models = [
    os.path.join(user_profile, r".cache\huggingface\hub\models--bert-base-multilingual-cased"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--xlm-roberta-base"),
    os.path.join(user_profile, r".cache\huggingface\hub\models--roberta-base")
]

# Remove cached models if they exist
for path in cached_models:
    if os.path.exists(path):
        shutil.rmtree(path)
        print(f"Removed cache: {path}")
    else:
        print(f"No cache found at: {path}")


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
import torch

print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())


# ==============================
# Imports
# ==============================
import ast
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report




# ==============================
# Set random seeds for reproducibility
# ==============================
# 💡 Added this block to ensure consistent results across runs
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

 
# Raw data: first 16 questions
data_16 = [
    {
        "question": "ایک بیٹری کس چیز کو چلانے کے لیے کیمیائی توانائی کو برقی توانائی میں تبدیل کرتی ہے؟",
        "choices": np.array(["اسفنج", "پتھر", "اسٹیپلر", "کیلکولیٹر"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "کیا تمام حشرات کو مکمل بالغ ہونے سے پہلے تبدیلی کے ہر مرحلے سے گزرنا پڑتا ہے؟",
        "choices": np.array(["یہ تمام", "حشرات زندہ پیدا ہوتے ہیں",
                             "پیوپا کا مرحلہ کبھی کبھی چھوڑ دیا جاتا ہے",
                             "پیوپا ایک لازمی مرحلہ ہے"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "C"
    },
    {
        "question": "کون سی توانائی ماحول کے لیے سب سے زیادہ موزوں ہے؟",
        "choices": np.array(["کوئلہ", "پیٹرولیم", "قدرتی گیس", "سورج کی روشنی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "ایک مگرمچھ کیا کھائے گا؟",
        "choices": np.array(["مکڑی", "کیکڑا", "سکویڈ", "تلاپیا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "حرارت کا ذریعہ ہو سکتا ہے",
        "choices": np.array(["ناک رگڑنا", "انٹارکٹیکا میں تیراکی کرنا", "برف کو چھونا",
                             "فریزر میں بیٹھنا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "فنچوں کو پسند ہے",
        "choices": np.array(["لطیفے", "جادو", "ایوری ون لوز ریمنڈ", "میٹھے سلطان"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "روشنی کس چیز میں سے نہیں گزر سکتی؟",
        "choices": np.array(["لکڑی", "پانی", "ہوا", "شیشہ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "ملچ کا ایک ڈھیر گل سڑ گیا ہے، اور جب اس کا سبب تلاش کیا جاتا ہے تو ایک باغبان کو نظر آتے ہیں",
        "choices": np.array(["بل کھاتے ہوئے جاندار", "لکھنے والے جاندار", "بڑے درخت", "جنگل کی آگ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "چاند طلوع ہوتا ہے",
        "choices": np.array(["ایک دہائی میں ایک بار", "سال میں 4 بار", "ہفتے میں 7 بار", "مہینے میں 1 بار"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "C"
    },
    {
        "question": "پہاڑ کی ڈھلوان پر چٹانوں کے ٹوٹنے کی سب سے زیادہ ممکنہ وجہ کیا ہے؟",
        "choices": np.array(["صنوبر کے درخت کی جڑیں", "حقیقی عرفان", "سمندری خارپشت", "ہیوی میٹل موسیقی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "کچھ پرندے مقامات تلاش کرتے ہیں",
        "choices": np.array(["زمینی نشانیوں سے", "سڑک کے نشانات سے", "انڈوں سے", "مقناطیسی نمونوں سے"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "کس سے بیج پھیلنے کا امکان ہے؟",
        "choices": np.array(["ایک کار", "سورج کی ایک کرن", "ایک وہیل", "ایک ہمنگ برڈ"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "ایک کار جو ایک ایسے وسیلے پر چلتی ہے جو بالآخر ختم ہو جائے گا",
        "choices": np.array(["ڈائنوسار کی باقیات", "پانی کی طاقت", "شمسی توانائی", "بجلی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "بجلی کے کنڈکٹرز ان طریقوں سے کام کرتے ہیں جیسے",
        "choices": np.array(["مائیکروویو کو چلانے کے لیے ساکٹ میں پلگ لگانا",
                             "صبح کے وقت کافی پاٹ آن کرنا",
                             "استعمال کے بعد اس بات کو یقینی بنانا کہ ہیئر ڈرائر ان پلگ ہے",
                             "کمرے کی لائٹیں جلانا"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "A"
    },
    {
        "question": "اگر کسی پرندے کی چونچ اپنے ساتھیوں سے بڑی ہے، تو غالب امکان ہے کہ اس کے تھے",
        "choices": np.array(["بغیر چونچ والی ماں", "لمبی سونڈ والا باپ",
                             "چھوٹی چونچوں والے والدین", "ایسی ہی چونچوں والے آباؤ اجداد"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "D"
    },
    {
        "question": "اگر کوئی شخص روزانہ منکے بیچ رہا ہے اور پھر کوئی بھی مزید منکے نہیں خریدتا، تو بیچنے والے کا کیا ہوگا؟",
        "choices": np.array(["وہ زیادہ پیسے کمائے گا", "اس کی آمدنی رک جائے گی",
                             "اس کا منافع بڑھ جائے گا", "اس کی آمدنی زیادہ ہو جائے گی"]),
        "labels": np.array(["A", "B", "C", "D"]),
        "answer": "B"
    }
]

# Convert to OpenPrompt InputExample format
train_dataset = []
guid = 0

for idx, item in enumerate(data_16):
    question = item["question"]
    labels = item["labels"]
    choices = item["choices"]
    correct_answer = item["answer"]

    for label, choice in zip(labels, choices):
        train_dataset.append(
            InputExample(
                guid=f"q{idx}_{label}",
                text_a=question,
                text_b=choice,
                label=1 if label == correct_answer else 0
            )
        )
        guid += 1

# Quick stats
print("Total examples:", len(train_dataset))
print("Correct examples:", sum(ex.label==1 for ex in train_dataset))
print("Incorrect examples:", sum(ex.label==0 for ex in train_dataset))




# ==============================
# Define Classes
# ==============================
classes = ["incorrect", "correct"]
label_map = {"incorrect": 0, "correct": 1}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")



template = ManualTemplate(
        text='سوال: {"placeholder":"text_a"} جواب: {"placeholder":"text_b"} یہ جواب {"mask"} ہے۔',
        tokenizer=tokenizer,
)


# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
        "incorrect": ["غلط", "نا درست"],
        "correct": ["درست", "صحیح"]
    },
    tokenizer=tokenizer,
)




# ==============================
# Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)
prompt_model.to(device)

# ==============================
# Training DataLoader
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=True
)

# ==============================
# Fine-Tune
# ==============================
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)
prompt_model.train()
for epoch in range(20):  # reduce epochs for testing
    torch.cuda.empty_cache()
    total_loss = 0
    for batch in train_loader:
        batch = {
            k: v.to(device) if torch.is_tensor(v) else v
            for k, v in batch.items()
        }
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'].long())
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

 

CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Total examples: 64
Correct examples: 16
Incorrect examples: 48


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
tokenizing: 64it [00:00, 3189.74it/s]


Epoch 1 Loss: 5.0778
Epoch 2 Loss: 4.6023
Epoch 3 Loss: 5.1812
Epoch 4 Loss: 5.1124
Epoch 5 Loss: 3.8831
Epoch 6 Loss: 3.4160
Epoch 7 Loss: 3.1498
Epoch 8 Loss: 3.1425
Epoch 9 Loss: 3.0782
Epoch 10 Loss: 3.1835
Epoch 11 Loss: 2.2595
Epoch 12 Loss: 1.5400
Epoch 13 Loss: 0.7323
Epoch 14 Loss: 0.8203
Epoch 15 Loss: 0.3788
Epoch 16 Loss: 0.6603
Epoch 17 Loss: 0.1897
Epoch 18 Loss: 0.6692
Epoch 19 Loss: 0.1827
Epoch 20 Loss: 0.4518


In [3]:
csv_path = r"C:\Users\stdFurqan\Desktop\paft\QA\openbookqa_urdu_test_cleaned.csv"
df = pd.read_csv(csv_path)

eval_dataset = []

for i, row in df.iterrows():
    question = row['urdu_question_stem']
    choices_dict = eval(row['clean_choices'])  # {'label': [...], 'text': [...]}

    labels = choices_dict['label']
    texts = choices_dict['text']
    answer = row['answerKey']

    for label, choice_text in zip(labels, texts):
        eval_dataset.append(
            InputExample(
                guid=f"eval_{i}_{label}",  # keeps question id
                text_a=question,
                text_b=choice_text,
                label=1 if label == answer else 0
            )
        )

print("✅ Total eval examples:", len(eval_dataset))
print("✅ Total questions:", len(df))


✅ Total eval examples: 3968
✅ Total questions: 992


In [4]:
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,              # single fixed template for eval
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)


prompt_model.eval()

question_scores = {}   # qid -> [(score, gold_label)]
example_ptr = 0        # pointer over eval_dataset

with torch.no_grad():
    for batch in eval_loader:
        batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}

        logits = prompt_model(batch)      # [B, 2]
        pos_scores = logits[:, 1]          # confidence of "correct" class

        for score, label in zip(pos_scores.cpu(), batch['label'].cpu()):
            # extract question id from guid: eval_<qid>_<choice>
            qid = eval_dataset[example_ptr].guid.split("_")[1]

            if qid not in question_scores:
                question_scores[qid] = []

            question_scores[qid].append((score.item(), label.item()))
            example_ptr += 1





correct = 0
total = 0

for qid, options in question_scores.items():
    # pick option with highest score
    _, predicted_label = max(options, key=lambda x: x[0])

    if predicted_label == 1:
        correct += 1
    total += 1

qa_accuracy = correct / total
print(f"\n📊 QA Accuracy: {qa_accuracy:.4f}")


tokenizing: 3968it [00:01, 2674.26it/s]



📊 QA Accuracy: 0.2641
